In [2]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [3]:
regex = pd.read_csv(r"keywords.csv") #(r"regex.txt")

In [4]:
k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == "case"):
        p_list.append(re.compile(k[i]))
    else:
        p_list.append(re.compile(k[i], re.IGNORECASE))
print(len(p_list))

36


In [5]:
john_hsu = pd.read_csv(r"C:\users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Bert\john_hsu_mapping.csv")

In [6]:
john_hsu = john_hsu[john_hsu["NoteTXT"].notna()]

In [7]:
john_hsu = john_hsu.reset_index(drop = True)

In [8]:
l = []
for note in tqdm(john_hsu["NoteTXT"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    l.append(str(curr))
    
john_hsu["regex_matches"] = l

100%|█████████████████████████████████████████████████████████████████████████| 387088/387088 [16:43<00:00, 385.86it/s]


In [9]:
john_hsu = john_hsu[john_hsu["regex_matches"] != "[]"]

In [10]:
locations = []
for r in tqdm(john_hsu['NoteTXT']):
    curr_loc = []
    for p in p_list:
         for match in re.finditer(p ,r):
            curr_loc.append(match.span())
            
    if (curr_loc == []):
        print("STOP")
        break
    locations.append(curr_loc)
    
john_hsu["regex_location"] = locations

100%|███████████████████████████████████████████████████████████████████████████| 46650/46650 [06:04<00:00, 127.99it/s]


In [11]:
john_hsu = john_hsu.reset_index(drop = True)

In [39]:
def create_context_windows(i):
    merged = []
    context_length_one_direction = 100
    
    for j in range(len(john_hsu["regex_location"][i])):
        start = john_hsu["regex_location"][i][j][0]
        end = john_hsu["regex_location"][i][j][1]
    
        prune_start = False
        prune_end = False
        if (start - context_length_one_direction <= 0):
            start = 0
            prune_start = True
        if (end + context_length_one_direction >= len(john_hsu["NoteTXT"]) - 1):
            end = len(john_hsu["NoteTXT"]) - 1
            prune_end = True

        if (prune_start and prune_end):
            merged.append((start, end))
        else:
            if (prune_start):
                merged.append((start, end + context_length_one_direction))
            if (prune_end):
                merged.append((start - context_length_one_direction, end))

        if (not prune_start and not prune_end):
            merged.append((start - context_length_one_direction, end + context_length_one_direction))
    
    return merged

In [40]:
merged_locs = []
for i in tqdm(range(len(john_hsu))):
     merged_locs.append(create_context_windows(i))

john_hsu["merged_row_location"] = merged_locs

100%|█████████████████████████████████████████████████████████████████████████| 46700/46700 [00:03<00:00, 14640.43it/s]


In [41]:
def merge(intervals):  
    ans = []
    i = 1
    intervals.sort()
    tempans = intervals[0]
    
    while i < len(intervals):
        #check intersection point
        if tempans[0] <= intervals[i][1] and tempans[1] >= intervals[i][0]:
            newtempans = [min(tempans[0] , intervals[i][0]) , max(tempans[1] , intervals[i][1])]
            tempans = newtempans
            i += 1
        else:
            ans.append(tempans)
            tempans = intervals[i]
            i += 1
            
    ans.append(tempans)
    return ans

In [42]:
merged_locs = []
for i in tqdm(range(len(john_hsu))):
     merged_locs.append(merge(john_hsu["merged_row_location"][i]))

john_hsu["merged_row_location"] = merged_locs        

100%|█████████████████████████████████████████████████████████████████████████| 46700/46700 [00:00<00:00, 60573.26it/s]


In [147]:
def pull_context_windows(i, col):
    sequence = ""
    
    for j in range(len(john_hsu[col][i])):
        if j > 0 and sequence != "":
            sequence += " ----- "
            
        start = john_hsu[col][i][j][0]
        end = john_hsu[col][i][j][1]
        
        sequence += john_hsu["NoteTXT"][i][start : end]
    
    return sequence

In [44]:
seqs = []

for i in tqdm(range(len(john_hsu))):
    seqs.append(pull_context_windows(i, "merged_row_location"))

john_hsu["regex_sent"] = seqs

100%|█████████████████████████████████████████████████████████████████████████| 46700/46700 [00:02<00:00, 18045.67it/s]


In [168]:
john_hsu["padded_merged_regex_location"] = john_hsu["merged_row_location"].copy()

for i in tqdm(range(len(john_hsu))):
    john_hsu["padded_merged_regex_location"][i] = list(eval(john_hsu["padded_merged_regex_location"][i]))

100%|██████████████████████████████████████████████████████████████████████████| 46700/46700 [00:37<00:00, 1244.89it/s]


In [200]:
def generate_padded_context_windows(i):
    if (john_hsu["token_length"][i] <= 400):
        locs = list(eval(john_hsu["merged_row_location"][i]))
        length = len(locs)
        start = locs[0][0] 
        end = locs[length - 1][1]
        
        #print(start, end)
                
        seq_length = 1000
        length_to_add_one_side = int((seq_length - john_hsu["sequence_length"][i]) / 2)
        
        # print(length_to_add_one_side, john_hsu["sequence_length"][i])
        
        prune_start = False
        prune_end = False
        if (start - length_to_add_one_side <= 0):
            start = 0
            prune_start = True
        if (end + length_to_add_one_side >= len(john_hsu["NoteTXT"]) - 1):
            end = len(john_hsu["NoteTXT"]) - 1
            prune_end = True
            
        # print(prune_start, prune_end)

        if (prune_start):
            end += length_to_add_one_side
            if (end + length_to_add_one_side <= len(john_hsu["NoteTXT"])):
                end += length_to_add_one_side
        if (prune_end):
            start -= length_to_add_one_side
            if (start - length_to_add_one_side >= 0):
                start -= length_to_add_one_side
        if (not prune_start and not prune_end):
            start -= length_to_add_one_side
            end += length_to_add_one_side
            
        locs = [list(x) for x in locs]
        locs[0][0] = start
        locs[length - 1][1] = end
        
        john_hsu["padded_merged_regex_location"][i] = locs

In [201]:
for i in tqdm(range(len(john_hsu))):
    generate_padded_context_windows(i)

100%|██████████████████████████████████████████████████████████████████████████| 46700/46700 [00:31<00:00, 1481.11it/s]


In [202]:
padded_seqs = []

for i in tqdm(range(len(john_hsu))):
    padded_seqs.append(pull_context_windows(i, "padded_merged_regex_location"))

john_hsu["padded_regex_sent"] = padded_seqs

100%|█████████████████████████████████████████████████████████████████████████| 46700/46700 [00:02<00:00, 16677.50it/s]


In [203]:
john_hsu['regex_matches'] = john_hsu['regex_matches'].astype(str)
john_hsu['regex_matches'] = john_hsu['regex_matches'].str.replace("\[\]", '')

In [204]:
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].astype(str)
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].str.replace('[', '')
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].str.replace(']', '')

In [205]:
john_hsu = john_hsu.dropna(subset=["padded_regex_sent"])
john_hsu['padded_regex_sent'] = john_hsu['padded_regex_sent'].str.replace('"', "'")

In [206]:
def note_preprocessing(note):
    note = re.sub(r'   • ', ' ; ', note)
    note = re.sub(r'[ ]{4,}', ' ', note)
    note = re.sub(r'  ', '\n', note)
    note = re.sub(r'    ', '\n\n', note)
    return note

In [207]:
for i in tqdm(range(len(john_hsu))):
    john_hsu.at[i, "padded_regex_sent_preprocessed"] = note_preprocessing(john_hsu.at[i, "padded_regex_sent"])

100%|█████████████████████████████████████████████████████████████████████████| 46700/46700 [00:03<00:00, 12788.21it/s]


In [208]:
john_hsu["sequence_length"] = john_hsu['padded_regex_sent_preprocessed'].astype(str).map(len)

In [209]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [210]:
def clinical_bert_tokenize(i):
    tokens = tokenizer.encode_plus(john_hsu["padded_regex_sent_preprocessed"][i], add_special_tokens = False, return_tensors = 'pt')

    return (len(tokens['input_ids'][0]))

In [211]:
token_lens = []

for i in tqdm(range(len(john_hsu))):
    token_lens.append(clinical_bert_tokenize(i))

john_hsu["padded_token_length"] = token_lens

100%|███████████████████████████████████████████████████████████████████████████| 46700/46700 [01:18<00:00, 594.00it/s]


In [213]:
len(john_hsu[john_hsu["padded_token_length"] > 512])

7734

In [7]:
john_hsu_no_padding = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Preprocessing\Getting_Data\Matches\john_hsu_dataset_matches_no_padding.csv")

In [8]:
len(john_hsu_no_padding[john_hsu_no_padding["token_length"] > 510])

7647

In [10]:
john_hsu = pd.read_csv(r"Matches/john_hsu_dataset_matches_padding.csv")

In [14]:
john_hsu = john_hsu.drop_duplicates(subset = ["padded_regex_sent_preprocessed", "NoteID"], keep = "last").reset_index(drop = True)

In [15]:
john_hsu["padded_token_length"].describe()

count    46664.000000
mean       382.321533
std        322.555366
min          0.000000
25%        255.000000
50%        306.000000
75%        380.000000
max       5160.000000
Name: padded_token_length, dtype: float64

In [12]:
john_hsu_no_padding = john_hsu_no_padding.drop_duplicates(subset = ["regex_sent_preprocessed", "NoteID"], keep = "last").reset_index(drop = True)

In [13]:
john_hsu_no_padding["token_length"].describe() # ax = john_hsu.plot.bar(y = "token_length", rot = 0)

count    46692.000000
mean       315.864024
std        347.917413
min          0.000000
25%        136.000000
50%        179.000000
75%        367.000000
max       5160.000000
Name: token_length, dtype: float64

In [223]:
john_hsu[:1000].to_csv(r"Matches/john_hsu_dataset_1000_padding.csv", index = False)

In [224]:
john_hsu.to_csv(r"Matches/john_hsu_dataset_matches_padding.csv", index = False)

In [225]:
patient_sequence_count = pd.DataFrame(columns = ["PatientID", "sequence_count"])

In [226]:
for i in range(len(john_hsu["PatientID"].unique())):
    count = len(john_hsu[john_hsu["PatientID"] == john_hsu["PatientID"].unique()[i]])
    patient_sequence_count.loc[i] = (john_hsu["PatientID"].unique()[i], count)

In [229]:
patient_sequence_count.to_csv(r"Matches/patient_level_sequence_data.csv", index = False)

In [3]:
patient_sequence_count = pd.read_csv(r"Matches/patient_level_sequence_data.csv")

In [4]:
patient_sequence_count["sequence_count"].describe()

count    952.000000
mean      49.016807
std       60.968176
min        1.000000
25%       10.000000
50%       27.000000
75%       65.250000
max      457.000000
Name: sequence_count, dtype: float64